In [ ]:
import math
import json
import requests
import itertools
import numpy as np
import time
import praw
import json
import copy
from datetime import datetime, timedelta

In [ ]:

def make_request(uri, max_retries = 5):
    current_try = 1
    def fire_away(uri,current_try):
        if current_try < max_retries:
            try:
                response = requests.get(uri)
                if response.status_code == 200:
                    jsonresp = json.loads(response.content)
                    if jsonresp['metadata']['timed_out'] == False and jsonresp['metadata']['shards']['failed']==0 and jsonresp['metadata']['shards']['successful']==jsonresp['metadata']['shards']['total']:
                        return json.loads(response.content)
                    else:
                        current_try+=1
                        time.sleep(1)
                        fire_away(uri,current_try)  
                else:
                    current_try+=1
                    time.sleep(1)
                    fire_away(uri,current_try)
            except:
                current_try+=1
                time.sleep(1)
                fire_away(uri,current_try)
        else:
            print('Cannot establish connection or wrong uri',uri)
            return None
    return fire_away(uri,current_try)

#make_request('https://api.pushshift.io/reddit/search/submission/?subreddit=PUBATTLEGROUNDS&user_removed=false&mod_removed=false&&fields=selftext,title,author,id&size=1&before=1544745600&after=1544659200&metadata=true')    

In [ ]:

def run_per_day(subreddit,start,end):
    uri ='https://api.pushshift.io/reddit/search/submission/?subreddit={}&user_removed=false&mod_removed=false&&fields=selftext,title,author,id&size=120&after={}&before={}&metadata=true'
    #uri = 'https://api.pushshift.io/reddit/search/submission/?subreddit={}&user_removed=false&mod_removed=false&size=100&fields=selftext,title,author,url,id,full_link&after={}&after{}'
    #rember size = 120
    uri=uri.format(subreddit,start,end)
    print('****************************'+str(start))
    response=make_request(uri)
    if response != None :
    #print(type(response))
        submissionsForToplevel = copy.deepcopy(response)
    #submissionsForAll = copy.deepcopy(response)
    #Only having two levels of comments 
        submissionList = []
        for submission in submissionsForToplevel['data']:
           submission['gencomments'] = makeObj(submission['id'])
           submissionList.append(submission)
       #print(type(response))
       #print(response)
        return {'jsonA':submissionList}
    else :
        return {'jsonA':None}
    #Having all comments
    '''submissionListWithCommentForrest = []
    for submission in submissionsForAll['data']:
       submission['gencomments'] = makeCompObj(submission['id'])
       submissionListWithCommentForrest.append(submission)'''
    #print('********' + str(start) + '*****************' )
    #print(submissionList)
    #print('---------------- whole count ------------------------')
    #print(submissionListWithCommentForrest)
    #Make json object for both lists
    '''jsonA = json.dumps(submissionList)
    jsonB = json.dumps(submissionListWithCommentForrest)
    with open(str(start)+'.json', 'w') as F:
        # Use the json dumps method to write the list to disk
        F.write(jsonA)
    with open(str(start)+'comp.json', 'w') as F:
        # Use the json dumps method to write the list to disk
        F.write(jsonB)'''
        #return {'jsonA':submissionList}
    #else :
        #return {'jsonA':None}
        
    
    
def makeIntervals(start,end):
    oneDay = 86400
    startTime = start
    endTime = start + oneDay
    #Make one mega file of 2 level comment submissions
    megaTwoLevel = []
    #Make one mega file of all level comment submissions
    #megaAllLevel = []
    #Doesnt work for 1 day gaps
    while((startTime - 1) != end):
        dictSubmissions = run_per_day('PUBATTLEGROUNDS',startTime,endTime)
        #print('Start Time ' + str(startTime))
        #print('endTime ' + str(endTime))
        jsonMA = dictSubmissions.get('jsonA')
        if jsonMA != None:
            megaTwoLevel.append(jsonMA)
        #jsonMB = dictSubmissions.get('jsonB')
        #megaAllLevel.append(jsonMB)
        
        startTime = (endTime + 1)
        endTime += oneDay
    
    convJsonA = json.dumps(megaTwoLevel)
    #convJsonB = json.dumps(megaAllLevel)
    #print(convJsonA)
    #print(convJsonB)
    #print(megaTwoLevel)
    with open('mega2level.json', 'w') as F:
        # Use the json dumps method to write the list to disk
        F.write(convJsonA)
    #with open('megaAlevel.json', 'w') as F:
        # Use the json dumps method to write the list to disk
        #F.write(convJsonB)
        
#makeIntervals(1544659200,1544745600)    
makeIntervals(1508889600,1540425600)

In [ ]:
reddit = praw.Reddit(client_id='w44W3IfIXbrE8w',
                     client_secret='j0el6C-wFuaSrMlPeA94aCXRWVQ',
                     redirect_uri='http://localhost:8080',
                     user_agent='testscript by UR')
print(reddit.auth.url(['identity'], '...', 'permanent'))
    

In [ ]:
def makeObj(subid):
    submission = reddit.submission(id=subid)
    submission.comments.replace_more(limit=None)
    commentForrest = ''
    for top_level_comment in submission.comments:
        commentForrest += ' ' + top_level_comment.body
        for second_level_comment in top_level_comment.replies:
            commentForrest += ' ' + second_level_comment.body
    #print(type(commentForrest))        
    return commentForrest

def makeCompObj(subid):
    submission = reddit.submission(id=subid)
    submission.comments.replace_more(limit=None)
    commentForrest = ''
    for comment in submission.comments.list():
        commentForrest += ' ' + comment.body
        
    return commentForrest
    

